In [ ]:
import geocoder

In [ ]:
key = 'pk.eyJ1IjoiZWFzaGVybWEiLCJhIjoiY2oxcW51Nzk2MDBkbTJxcGUxdm85bW5xayJ9.7mL0wQ7cjifWwt5DrXMuJA'
m = geocoder.mapbox('121 N LaSalle Dr, Chicago, IL 60602', key=key)

In [ ]:
key = 'pk.eyJ1IjoiZWFzaGVybWEiLCJhIjoiY2oxcW51Nzk2MDBkbTJxcGUxdm85bW5xayJ9.7mL0wQ7cjifWwt5DrXMuJA'
g = geocoder.google('121 N LaSalle Dr, Chicago, IL 60602')

In [ ]:
g = geocoder.google('121 N LaSalle Dr, Chicago, IL 60602')
e = geocoder.arcgis('121 N LaSalle Dr, Chicago, IL 60602')


In [21]:
import pandas as pd

df = pd.read_csv('./documenters_aggregator/local_outputs/chi_police_20180312_2010.csv')

In [53]:
test_addy = df['location_address'][0]

In [114]:
import geocoder

"""
This pipeline decorates items with latitude and longitude by geocoding their
addresses.
"""
class MapboxPipeline(object):
    """
    Stub pipeline to geocode using Mapbox.
    """
    def __init__(self):
        # the geocoder library. this way we can use other libraries if needed or write/extend our own
        self.engine = 'geocoder' 
        # api key, mapbox in this case. best practice would be to hide this
        self.key = 'pk.eyJ1IjoiZWFzaGVybWEiLCJhIjoiY2oxcW51Nzk2MDBkbTJxcGUxdm85bW5xayJ9.7mL0wQ7cjifWwt5DrXMuJA' #API 

    """
    Process an item.
    """
    def process_item(self, item, spider):
        if item['location']['coordinates']['latitude'] is None:
            response = self.get_geocoder_query('1811 N francisco ave, Chicago, Illinois, 60647')
            item['location']['coordinates']['latitude'] = response.lat
            item['location']['coordinates']['longitude'] = response.lng
            item['location']['url'] = response.url
        return item


    def get_geocoder_query(self, address=None, bbox=None):

        if self.engine == 'geocoder':
            provider = 'mapbox'
            params = {
            'engine': self.engine,
            'provider': provider,
            'key': self.key


            }
            query_params = {

                'address': address,
                'bbox': bbox # bounds to search in, useful if we dont have city/zip in address          
            }
            
#             query = "{engine}.{provider}('{address}', key='{key}')".format(**params)
            
            result = getattr(geocoder, provider)(query_params['address'], key = params['key'], **query_params)
            return result, str(query_params)
        return query



In [115]:
geo = MapboxPipeline()


In [139]:
test_raw = geo.get_geocoder_query(df['location_address'][0])
test_bbox = geo.get_geocoder_query(df['location_address'][0], bbox=[-87.940102,41.643921,-87.523987,42.023022])
test_city_state = geo.get_geocoder_query(df['location_address'][0] + ',Chicago,Illinois')

In [97]:
geo.get_geocoder_query(df['location_address'][0], bbox=[-87.940102,41.643921,-87.523987,42.023022])

(<[OK] Mapbox - Geocode #5 results>,
 "geocoder.mapbox('St. Ferdinand's 3115 N Mason', key='pk.eyJ1IjoiZWFzaGVybWEiLCJhIjoiY2oxcW51Nzk2MDBkbTJxcGUxdm85bW5xayJ9.7mL0wQ7cjifWwt5DrXMuJA')")

In [51]:
import json
json.dumps(test[0].geojson)

'{"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {"accuracy": "interpolated", "address": "3115 North Mason Avenue, Chicago, Illinois 60634, United States", "city": "Chicago", "country": "United States", "housenumber": "3115", "lat": 41.938369, "lng": -87.774988, "ok": true, "postal": "60634", "quality": 0.3333333333333333, "raw": {"id": "address.1218523901163610", "type": "Feature", "place_type": ["address"], "relevance": 0.3333333333333333, "properties": {}, "text": "North Mason Avenue", "place_name": "3115 North Mason Avenue, Chicago, Illinois 60634, United States", "center": [-87.774988, 41.938369], "geometry": {"type": "Point", "coordinates": [-87.774988, 41.938369], "interpolated": true}, "address": "3115", "context": [{"id": "neighborhood.298742", "text": "Belmont Central"}, {"id": "postcode.17261599636613900", "text": "60634"}, {"id": "place.7071500072701850", "wikidata": "Q1297", "text": "Chicago"}, {"id": "region.224711", "short_code": "US-IL", "wi

In [187]:
import folium 

bbox = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": 
[[[-87.940102,41.643921],[-87.523987,41.643921],[-87.523987,42.023022],[-87.940102,42.023022],[-87.940102,41.643921]]]
      }
    }
  ]
}



m = folium.Map(
    location=[42.023022, -87.523987],
    tiles='Mapbox Bright',
    zoom_start=1
)

folium.GeoJson(
    test_raw[0].geojson,
    name='test_raw'
).add_to(m)

folium.GeoJson(
    test_bbox[0].geojson,
    name='test_bbox'
).add_to(m)

folium.GeoJson(
    test_city_state[0].geojson,
    name='test_city_state'
).add_to(m)

folium.GeoJson(
    bbox,
    name='bbox'
).add_to(m)

folium.LayerControl().add_to(m)

m